In [ ]:
import torch
import numpy as np
import scipy.linalg
from numpy import linalg as LA


np.random.seed(0)
n = 2
def psd(n):
  S = np.random.randn(n,n)
  return S.T@S
r = 0.1
A = np.eye(n)


### for two player case, check the conditions from Rosen paper on the uniqueness of equilibrium point

In [ ]:

Q1 = np.diag([1.0, 0])
Q2 = np.diag([0, 1.0])
Q1 = torch.tensor(Q1).float()
Q2 = torch.tensor(Q2).float()

# apply system dynamics x[t+1] = (A-BK)x[t]
from torch.autograd import Variable

A = torch.tensor(np.eye(2)).float()
for j in range(1000):
    B = psd(2)
    if j % 100 == 0:
        print("=", end='')
    for k1_val in np.linspace(-5, 5, 10):
        for k2_val in np.linspace(-5, 5, 10):
            K = np.diag([k1_val, k2_val])
            if np.all(abs(LA.eigvals(A-B@K))<1):
                T = 10
                k1 = torch.tensor(k1_val, requires_grad=True)
                k2 = torch.tensor(k2_val, requires_grad=True)
                assert k1.requires_grad
                assert k2.requires_grad
                k1.retain_grad()
                k2.retain_grad()
                B = torch.tensor(B)
                def cost(k1,k2):
                    curr_x = torch.randn(2)
                    c1_list = []
                    c2_list = []
                    K = torch.tensor([[k1, 0], [0, k2]])
                    A_k = (A-B*K).float()
                    for t in range(T):
                        curr_x = A_k@curr_x
                        c1_list.append(curr_x.T@Q1@curr_x)
                        c2_list.append(curr_x.T@Q2@curr_x)
                    return torch.sum(torch.FloatTensor(c1_list)), torch.sum(torch.FloatTensor(c2_list))
                c1, c2 = cost(k1,k2)     
                du1k1 = torch.autograd.grad(c1, k1, retain_graph=True, create_graph=True)
                d2u1k1 = torch.autograd.grad(du1k1, k1, retain_graph=True)
                d2u1k2 = torch.autograd.grad(du1k1, k2, retain_graph=True)
                du2k2 = torch.autograd.grad(c2, k2, retain_graph=True, create_graph=True)
                d2u2k2 = torch.autograd.grad(du2k2, k2, retain_graph=True)
                d2u2k1 = torch.autograd.grad(du2k2, k1, retain_graph=True)
                J = np.array([[d2u1k1[0].item(), d2u1k2[0].item()], [d2u2k1[0].item(), d2u2k2[0].item()]])
                if all(LA.eigvals(J)>0) == False:
                  print('*')
    
